In [21]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import os

base_dir = 'data/현대 국어 사용 빈도 조사 2'

sorted(os.listdir(base_dir))

['구통계.txt',
 '구통계.xlsx',
 '어미통계.txt',
 '어미통계.xlt',
 '음절통계.txt',
 '음절통계.xlt',
 '일반어휘통계.txt',
 '일반어휘통계.xlsx',
 '자모통계.txt',
 '자모통계.xlt',
 '조사통계.txt',
 '조사통계.xlt',
 '현대 국어 사용 빈도 조사 2(2005).PDF']

In [20]:
Phrases = pd.read_excel(os.path.join(base_dir, '구통계.xlsx')) #.sort_values('빈도', ascending=False)[:50]
Vocab = pd.read_excel(os.path.join(base_dir, '일반어휘통계.xlsx')) #.sort_values('빈도', ascending=False)[:50]
Letters = pd.read_excel(os.path.join(base_dir, '자모통계.xlt')) #.sort_values('빈도', ascending=False)[:20]
Auxiliaries = pd.read_excel(os.path.join(base_dir, '조사통계.xlt')) #.sort_values('빈도', ascending=True)[:20]
Syllables = pd.read_excel(os.path.join(base_dir, '음절통계.xlt')) #.sort_values('빈도', ascending=True)[:20]

def getCharNames(s):
    return [unicodedata.name(c) for c in s]

def hasReduplication(s):
    return pd.Series(c for c in s).value_counts().max() > 1


ReduplicatedAdverbs = Vocab[(Vocab.품사=='부')&(Vocab.어휘.apply(hasReduplication))]

In [96]:
pd.Series([c for s in list(Vocab.어휘.apply(getCharNames)) for c in s]).value_counts()[:20]

DIGIT ZERO               23260
HANGUL SYLLABLE DA       17464
DIGIT ONE                 9282
HANGUL SYLLABLE HA        8714
DIGIT TWO                 6794
HANGUL SYLLABLE I         3796
HANGUL SYLLABLE JEOG      3719
HANGUL SYLLABLE SA        3270
HANGUL SYLLABLE GI        3133
HANGUL SYLLABLE RI        3055
HANGUL SYLLABLE JA        2883
DIGIT THREE               2816
HANGUL SYLLABLE JI        2800
HANGUL SYLLABLE SI        2634
HANGUL SYLLABLE DOE       2386
HANGUL SYLLABLE HWA       2377
HANGUL SYLLABLE DAE       2309
HANGUL SYLLABLE SU        2265
HANGUL SYLLABLE SEONG     2180
HANGUL SYLLABLE JANG      2032
dtype: int64

In [93]:
ReduplicatedAdverbs.sort_values('빈도', ascending=False)[:30]

,순위,빈도,어휘,풀이,품사
40921,551,661,스스로,NaN,부
740,603,594,각각01,各各,부
60924,709,473,점점01,漸漸,부
68302,803,370,천천히,NaN,부
12572,982,190,내내01,NaN,부
36436,991,181,서서히,徐徐-,부
63284,1047,125,종종04,種種,부
77342,1047,125,하하01,NaN,부
79335,1075,97,헤헤,NaN,부
56481,1087,85,일일이02,一一-,부


In [71]:
Vocab[(Vocab.품사 == '부')]

,순위,빈도,어휘,풀이,품사
9,1168,4,가가호호,家家戶戶,부
71,1136,36,가급적,可及的,부
72,1116,56,가까스로,NaN,부
74,940,232,가까이,NaN,부
80,798,375,가끔,NaN,부
...,...,...,...,...,...
82463,1139,33,힐끗,NaN,부
82465,1169,3,힐끗힐끗,NaN,부
82474,1100,72,힘껏,NaN,부
82486,1118,54,힘없이,NaN,부


In [28]:
Vocab['품사'].value_counts()

명    58605
동    13292
형     3411
부     3103
관     2570
의      542
감      384
수      285
대      162
접       76
보       66
지        5
Name: 품사, dtype: int64

In [69]:
Vocab[(Vocab.품사 == '부')&(Vocab.빈도 > 10)].sample(10).sort_values('빈도')

,순위,빈도,어휘,풀이,품사
79208,1161,11,헐레벌떡,NaN,부
53860,1161,11,은은히02,隱隱-,부
59557,1158,14,저토록,NaN,부
77164,1157,15,하릴없이,NaN,부
73318,1153,19,톡02,NaN,부
12075,1152,20,난데없이,NaN,부
77074,1129,43,하나같이,NaN,부
14938,1105,67,단숨에,單-,부
25474,1083,89,바짝,NaN,부
54637,1042,130,이내03,NaN,부


In [67]:
Phrases[Phrases['빈도'] >= 10].sample(20)

,순위,빈도,항목,풀이
3449,67,15,외국 자본,外國資本
4272,65,17,전자 우편,電子郵便
2280,67,15,사회 보장,社會保障
5035,45,37,케이블 티브이,cable TV
5415,58,24,학교 교육,學校敎育
1160,51,31,농업 경영,農業經營
4038,70,12,자본 축적,資本蓄積
4354,52,30,정보 산업,情報産業
3618,71,11,위화도 회군,威化島回軍
4004,66,16,자기 자본,自己資本
